# Assessing Data Quality

In [ ]:
import pandas as pd
import random as random
import numpy as np
df = pd.read_csv("dataset/TC-dataset.csv", sep="\t", index_col=0, decimal=",")
df.info()

## Data Selection
In questa sezione a partire dai dati originali si effettua una selezione dei dati in modo tale da eliminare tutti quelli semanticamente errati. Queste eliminazioni provengono da alcune assunzioni effettuate sui dati e sulla loro provenienza

### Provenienza dei dati
I dati provengono dal dataset https://archive.ics.uci.edu/ml/datasets/Online+Retail Nella fase di data semantics abbiamo già precisato alcune informazioni:
Si tratta di un sito e-commerce Inglese. Vende principalmente regali per tutte le occasioni e molti dei clienti sono grossisti. Nella colonna Sale è presente il prezzo per singolo articolo quindi equivale a quantità=1. Il prezzo è in sterline. Secondo la descrizione del dataset quando in basket id abbiamo una C iniziale si tratta di ordini cancellati.

### Trattamento Ordini cancellati
Per quanto ci riguarda gli ordini cancellati non sono significativi per noi per caratterizzare i clienti. Di conseguenza possiamo eliminarli. Tutti gli ordini da cancellare hanno anche quantità<=0

In [ ]:
print("Ordini da cancellare con \"C\": ",len(df[df["BasketID"].str.contains("C")]))
print("Ordini da cancellare con Qta<=0: ",len(df[df["Qta"]<=0]))
print("Record da cancellare con \"C\" con quantità<=0: "+str(len(df[(df["Qta"]<=0) & (df["BasketID"].str.contains("C"))])))

Ogni ordine cancellato potrebbe averne un corrispondente positivo quindi prima di eliminarli verifichiamo quindi se troviamo degli ordini opposti, prediamo tutti quelli negativi e verifichiamo.

In [ ]:
list_baskC=df[df["BasketID"].str.contains("C")]
count=0

for index, row in list_baskC.iterrows():

    local_search=df[(df["CustomerID"]==row['CustomerID'])&(df["Sale"]==row['Sale'])&(df["Qta"]==-row['Qta'])&(df["ProdID"]==row['ProdID'])]

    if len(local_search)>0:# In questo caso vuol dire che ci sono due ordini opposti
        count+=1
        ##print(index)
        ##print(df.loc[index])
        ##print(local_search.index[0])
        ##print(df.loc[local_search.index[0]])
        ##print("_________________________")
        df.drop([index,local_search.index[0]], inplace=True)
        
        
        
print("Record cancellati: ",2*count)
print("Nuova grandezza del dataset: ",len(df))

Nell'analisi non abbiamo considerato che non per forza un ordine cacellato ha la stessa quantità, potrebbe essere su un ordine di 50 elementi solo 20 per esempio siano stati cancellati. Tuttavia abbiamo semplificato il problema per evitare analisi troppe complicate. Magari con lo sviluppo del progetto potremo valutare di migliorare questa analisi. Vediamo quanti record rimangono:

In [ ]:
print("Record ancora da eliminare: ",len(df[df["BasketID"].str.contains("C"]))

Possiamo prendere i rimanenti e cercare nel dataset se c'è un altro ordine positivo con una quantità maggiore del valore assoluto in modo tale da elimiare un po' di quantità. Quindi rispetto a prima rimangono da fare i record in cui il valore assoluto della quantità negativa non ha un record >=

In [ ]:
remaining_order=df[df["BasketID"].str.contains("C")]
count=0

for index,row in remaining_order.iterrows():
    transaction_retrived=df[(df["CustomerID"]==row["CustomerID"])&(df["ProdID"]==row["ProdID"])&(df["Sale"]==row["Sale"])&(df["Qta"]>-row["Qta"])].sort_values(by=['BasketDate'])
    if len(transaction_retrived)>0:
        df.drop([index], inplace=True)
        transaction_retrived.Qta.iloc[0]=transaction_retrived.Qta.iloc[0]-row["Qta"]
        count+=1
print("Record eliminati: ",count)

In [ ]:
len(df[df["BasketID"].str.contains("C")])

Oltre a quelli opposti verifichiamo che non ci siano altri ordini cancellati. In questo caso li eliminamo comunque perchè potrebbero essere errori, anche se non hanno un corrispettivo positivo. Oppure potrebbero essere ordini cancellati corrispondenti a ordini fatti prima del dicembre 2010.

In [ ]:
print("Record da eliminare: ",len(df[df["BasketID"].str.contains("C")]))
df=df[df["BasketID"].str.contains("C") == False]

### Quantità negative con ordini non cancellati
Abbiamo già constatato che gli ordini con la C sono cancellati e abbiamo già deciso di cancellarli. Consideriamo ora quelli con solo le quantità negative.

In [ ]:
print("Record con quantità <=0: ", len(df[df["Qta"]<=0]))
print("Record con quantità <=0 e Sale<=0: ", len(df[(df["Qta"]<=0) & (df["Sale"]<=0)]))
print(df[df["Qta"]<=0].ProdDescr.unique())


Abbiamo solamente 668 transazioni in cui abbiamo la quantità negativa e non appartengono a ordini cancellati. Inoltre in questi 668 tutti hanno sale<=0.  Dalle descrizioni fornite la maggior parte sono quindi articoli persi oppure danneggiati o eventuali errori. 
Eliminare anche questi record, in futuro possiamo pensare a qualche indicatore che possa tenere conto di questi articoli. In ogni caso sono solo 668 elementi.

In [ ]:
df=df[df["Qta"]>0]

### Sale <=0 e quantità >=0
Abbiamo trattato le quantità <=0. Altri valori particolari sono quelli relativi al Sale. In particolare ci sono alcuni sale che sono <=0.

In [ ]:
print("Sale<=0: ",len((df[df["Sale"]<=0])))
print("Sale<0: ",len((df[df["Sale"]<0]))," record\n", df[df["Sale"]<0])

Esistono solo 2 record con sale < 0 che eliminati in quanto non rappresentano una informazione relativa ai customer ma sono relativi all'azienda.

611 Record hanno sia il sale > 0 e la quantità > 0, essendo il nostro clustering fatto sui clienti e non sui prodotti potrebbe fuoriviante avere alcuni record con sale = 0.
Potrebbero essere omaggi per esempio e non avrebbe senso considerarli.
Alcuni con il sale = 0 hanno quantità molto alte anche di 300 unità, in questo caso sarebbe anche difficile considerarli omaggi a meno che non siano per grossisti.

Dato che stiamo performando una analisi sui customer trovo poco significativo considerare gli ordini "omaggio" che sono una registrazione che viene fatta dall'azienda per tenerne traccia

Altra possibilità fare il clustering con questi e vedere se effettivamente ci deviano il clustering, sono comunque pochi record.

In [ ]:
df=df[df["Sale"]>=0]

## Missing Values
### Missing Values CustomerID

In [ ]:
print("Numero di CustomerID Null: " + str(len(df[df["CustomerID"].isnull()])))

In [ ]:

#Generazione dei nuovi CustomerID
basket_list_customer_null=df[df["CustomerID"].isnull()].BasketID.unique()
new_customer_per_basket_list=random.sample(range(1, 100000), len(basket_list_customer_null))

#Inserimento dei customer id con il contrassegno N per il riconoscimento
for i, elem in enumerate(basket_list_customer_null):
    df["CustomerID"][df.BasketID==elem]=str(new_customer_per_basket_list[i])+"N"

In [ ]:
print("Numero di CustomerID Null: " + str(len(df[df["CustomerID"].isnull()])))

### Country Missing Values

In [ ]:
#Numero di righe con country Unspecified
print("Numero di Unspecified: " + str(len(df[df["CustomerCountry"].str.contains("Unspecified")])))

#Numero di righe con country European Community
print("Numero di European Community: " + str(len(df[df["CustomerCountry"].str.contains("European Community")])))

In [ ]:
#Moda di CustomerCountry
moda = df['CustomerCountry'].mode()[0]
print("Moda: " + str(moda))


df["CustomerCountry"].replace({"Unspecified": moda}, inplace=True)
df["CustomerCountry"].replace({"European Community": moda}, inplace=True)

print("Numero di Unspecified: " + str(len(df[df["CustomerCountry"].str.contains("Unspecified")])))
print("Numero di European Community: " + str(len(df[df["CustomerCountry"].str.contains("European Community")])))

### ProdDesc

In [ ]:
len(df[df['ProdDescr'].isnull()])

La mancanza di valori per la descrizione del prodotto non ci interessa più di tanto, l'identificativo del prodotto è più rilevante, sono due attributi doppi. Per recuperalo potremo controllare se nei sample con gli stessi Productid di quelli mancanti ci sono i dati

In [ ]:
retrievable_count=0
error_count=0
for elem in df[df['ProdDescr'].isnull()].ProdID.unique():
    df_loc_equal_ProdID=df[df['ProdID']==elem]
    
    if len(df_loc_equal_ProdID[df_loc_equal_ProdID['ProdDescr'].notnull()])>0:
        # Recupero descrizione a partire dal primo prodotto della lista
        retrived_ProdDescr=df_loc_equal_ProdID[df_loc_equal_ProdID['ProdDescr'].notnull()].iloc[0].ProdDescr

        #Vediamo se le descrizioni sono tutte uguali saranno nan oppure una descrizione del prodotto
        if(len(df_loc_equal_ProdID['ProdDescr'].unique())>2):
            error_count+=1

        #DEBUG
        #print("ProdId: "+elem+" ProdDescr: "+retrived_ProdDescr)
        
        retrievable_count+=1
print("Totale elementi sostituibili: "+str(retrievable_count))
print("Errori trovati: "+str(error_count))

Del dataframe creato si possono sostituire 541 elementi su 753 ma di questi 76 non sono coerenti tra di loro (stesso ProdID ma diversa ProdDescr). In ogni caso la descrizione nel prodotto è inutile quindi possiamo evitare di processarla e droppare la colonna nella data transformation

## Outliers

## Salvatagio Dataframe

In [ ]:
df.to_csv('dataset/DQ-dataset.csv', sep='\t',decimal=",")